## Treinamento do modelo

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks, initializers, losses
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0

In [ ]:
IMG_SIZE   = (224, 224)
BATCH_SIZE = 32

# variedade de exemplos
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,         # rotações até 30°
    width_shift_range=0.2,     # deslocamentos horizontais
    height_shift_range=0.2,    # deslocamentos verticais
    zoom_range=0.2,            # zoom in/out
    brightness_range=(0.8,1.2),# variação de brilho
    horizontal_flip=True,      # flip horizontal
    validation_split=0.2       # 20% para validação
)
val_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

In [ ]:
train_gen = train_datagen.flow_from_directory(
    'data/16_animes',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True
)
val_gen = val_datagen.flow_from_directory(
    'data/16_animes',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)
num_classes = train_gen.num_classes

In [ ]:
base_model = EfficientNetB0(
    include_top=False,
    weights='imagenet',
    input_shape=IMG_SIZE + (3,)
)
base_model.trainable = False 

In [ ]:
initializer = initializers.HeUniform()
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(), 
    layers.BatchNormalization(), 
    layers.Dropout(0.3), 
    layers.Dense(256, 
                 activation='relu', 
                 kernel_initializer=initializer),
    layers.BatchNormalization(),
    layers.Dropout(0.4),
    layers.Dense(num_classes, activation='softmax')
])

loss_fn = losses.CategoricalCrossentropy(label_smoothing=0.1)

lr_schedule = optimizers.schedules.CosineDecay(
    initial_learning_rate=1e-4,
    decay_steps=1000
)
optimizer = optimizers.Adam(learning_rate=1e-4)

In [ ]:
model.compile(
    optimizer=optimizer,
    loss=loss_fn,
    metrics=['accuracy']
)
model.summary()

early_stop = callbacks.EarlyStopping(
    monitor='val_loss', patience=5, restore_best_weights=True, verbose=1
)
reduce_lr = callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.5, patience=3, verbose=1
)
checkpoint = callbacks.ModelCheckpoint(
    'best_model.h5', monitor='val_loss', save_best_only=True, verbose=1
)

In [ ]:
EPOCHS_HEAD = 20
history1 = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS_HEAD,
    callbacks=[early_stop, reduce_lr, checkpoint]
)

In [ ]:
base_model.trainable = True
total_layers = len(base_model.layers)
freeze_until = int(total_layers * 0.3)
for layer in base_model.layers[:freeze_until]:
    layer.trainable = False

model.compile(
    optimizer=optimizer,
    loss=loss_fn,
    metrics=['accuracy']
)

In [ ]:
EPOCHS_FINE = 20
history2 = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS_HEAD + EPOCHS_FINE,
    initial_epoch=history1.epoch[-1],
    callbacks=[early_stop, reduce_lr, checkpoint]
)

In [ ]:
model.load_weights('best_model.h5')
loss, acc = model.evaluate(val_gen)
print(f'Val Loss: {loss:.4f} — Val Accuracy: {acc:.4%}')

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(history1.history['accuracy']  + history2.history['accuracy'],  label='Train Acc')
plt.plot(history1.history['val_accuracy'] + history2.history['val_accuracy'], label='Val Acc')
plt.xlabel('Época')
plt.ylabel('Acurácia')
plt.legend()
plt.tight_layout()
plt.show()

Found 2057 images belonging to 7 classes.
Found 513 images belonging to 7 classes.
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb0 (Functional)  (None, 7, 7, 1280)       4049571   
                                                                 
 global_average_pooling2d_1   (None, 1280)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 batch_normalization_3 (Batc  (None, 1280)             5120      
 hNormalization)                                                 
                                                                 
 dropout_3 (Dropout)         (None, 1280)              0         
                                                                 
 dense_3 (Dense)             (None, 256)               327936    
                                     

KeyboardInterrupt: 